# Connected Agents

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole, ConnectedAgentTool

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [10]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup Connected Agent Tools

In [ ]:
search_agent = project_client.agents.get_agent("asst_kZB1IGPvXK5TBfBX1STUusjE")
report_agent = project_client.agents.get_agent("asst_jJnvQxXcmBwWBXTvCFOsNEhQ")
validation_agent = project_client.agents.get_agent("asst_r8FtTSPrDwNPsgN3radMv3nJ")

search_connected_agent = ConnectedAgentTool(
        id=search_agent.id, name="search_health_plans", description="Searches for information about health plans."
    )
report_connected_agent = ConnectedAgentTool(
        id=report_agent.id, name="create_report", description="Generates reports on the health plan."
    )
validation_connected_agent = ConnectedAgentTool(
        id=validation_agent.id, name="validate_report", description="Validates the health plan report."
    )

## Create the agent

In [16]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-healthreport-agent",  # Name of the agent
    instructions="""
        You are an agent designed to create detailed reports about health plans. 
        The user will provide the name of a health plan and you will create a detailed report about that health plan. 
        You will also need to validate that the report meets requirements. 
        Call the appropriate agents to help write the report. 
        
        Do not write the report on your own. 
        Your role is to be an orchestrator who will call the appropriate functions provided to you. 
        Each function that you have available is an agent that can accomplish a specific task. 
        
        Here are descriptions of the plugins you have available:
            - Report Agent: An agent that writes detailed reports about health plans.
            - Search Agent: An agent that searches health plan documents.
            - Validation Agent: An agent that runs validation checks to ensure the generated report meets requirements. 
            
        Your response must contain the final approved report.
    """,  # Instructions for the agent
    tools=search_connected_agent.definitions + report_connected_agent.definitions + validation_connected_agent.definitions,  # Tools available to the agent
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_2wzzPC2yAfKrozDYczmbM9u5


## Create thread

In [17]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_L1T0EDYT9EcKLLszJzjveMaz


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [18]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [19]:
user_input = "Create a report about the Northwind Health Plus"
run_agent(user_input)

Created message, ID: msg_KAVIuh9Q3SWLzWqwvV19DVur
Run finished with status: completed
Agent Response: Here is the validated report on Northwind Health Plus:

# Northwind Health Plus Report

## Coverage Details

### Inpatient Coverage
- **Hospital Stays:** Covers a broad range of inpatient services, including room and board, nursing services, and medications administered during the hospital stay.
- **Surgical Procedures:** Comprehensive coverage for elective and emergency surgeries performed during inpatient stays.

### Outpatient Coverage
- **Medical Appointments:** Includes primary care visits, specialist consultations, and preventive care.
- **Diagnostic Services:** Coverage for laboratory tests, X-rays, MRI, and other diagnostic procedures.
- **Therapies:** Includes physical therapy, occupational therapy, and speech therapy on an outpatient basis.

### Prescription Drugs
- **Formulary:** Covers a wide range of prescription medications listed in the plan's formulary. 
- **Tiers:** Me

In [17]:
user_input = "What is the weather in Chicago?"
run_agent(user_input)

Created message, ID: msg_43SnhCi6txThfti1110f8Qyu
Run finished with status: completed
Agent Response: Here is the weather information for Chicago:

| Weather Element       | Value                            |
|-----------------------|----------------------------------|
| Condition             | Mostly clear                     |
| Temperature           | 23.0°C                           |
| Feels Like            | 23.0°C                           |
| Dew Point             | 20.0°C                           |
| Humidity              | 83%                              |
| Barometric Pressure   | 1015.0 hPa                       |
| Visibility            | 16 km                            |
| Wind Direction        | 217°                             |
| Wind Speed            | 7.0 km/h                         |
| Wind Gust             | 10.0 km/h                        |
| UV Index              | 0.0 (Low)                        |
| Air Quality Index     | 87 (Moderate air quality)        

## Cleanup resources

In [20]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
